In [ ]:
!pip install pandas pyarrow


In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
df = pd.read_parquet('/content/yellow_tripdata_2023-01.parquet')
# df = pd.read_parquet('nyctaxidata.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Lets start data modelling

fact table

datetime_dim





















In [ ]:
df.info()
#data types make sense

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [ ]:
#drop duplicated values so the dataset is clean
df = df.drop_duplicates().reset_index(drop=True)

# df['trip_id'] = df.index

In [ ]:
#create an index called trip_id

df['trip_id'] = df.index
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_id
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,1
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,2
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,3
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,4


In [ ]:
#create the datetime dimension table

#pick up datetime
datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday


#drop off datetime
datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

#create a date_time id
datetime_dim['datetime_id'] = datetime_dim.index

#change the order of the columns
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

datetime_dim.head()



,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2023-01-01 00:32:10,0,1,1,2023,6,2023-01-01 00:40:36,0,1,1,2023,6
1,1,2023-01-01 00:55:08,0,1,1,2023,6,2023-01-01 01:01:27,1,1,1,2023,6
2,2,2023-01-01 00:25:04,0,1,1,2023,6,2023-01-01 00:37:49,0,1,1,2023,6
3,3,2023-01-01 00:03:48,0,1,1,2023,6,2023-01-01 00:13:25,0,1,1,2023,6
4,4,2023-01-01 00:10:29,0,1,1,2023,6,2023-01-01 00:21:19,0,1,1,2023,6


In [ ]:
#create the passenger count dimension
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]
passenger_count_dim.head()

,passenger_count_id,passenger_count
0,0,1.0
1,1,1.0
2,2,1.0
3,3,0.0
4,4,1.0


In [ ]:
#trip distance dimesnion table
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]
trip_distance_dim.head()

,trip_distance_id,trip_distance
0,0,0.97
1,1,1.10
2,2,2.51
3,3,1.90
4,4,1.43


In [ ]:
#as per the data dictionary
# develop the rate code dimension table

rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]
rate_code_dim

,rate_code_id,RatecodeID,rate_code_name
0,0,1.0,Standard rate
1,1,1.0,Standard rate
2,2,1.0,Standard rate
3,3,1.0,Standard rate
4,4,1.0,Standard rate
...,...,...,...
3066761,3066761,NaN,NaN
3066762,3066762,NaN,NaN
3066763,3066763,NaN,NaN
3066764,3066764,NaN,NaN


In [ ]:
#payment type dimension table

payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

payment_type_dim

,payment_type_id,payment_type,payment_type_name
0,0,2,Cash
1,1,1,Credit card
2,2,1,Credit card
3,3,1,Credit card
4,4,1,Credit card
...,...,...,...
3066761,3066761,0,NaN
3066762,3066762,0,NaN
3066763,3066763,0,NaN
3066764,3066764,0,NaN


In [ ]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,2,0,0,0,0,N,0,9.30,1.00,0.5,0.00,0.0,1.0,14.30
1,1,2,1,1,1,1,N,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90
2,2,2,2,2,2,2,N,2,14.90,1.00,0.5,15.00,0.0,1.0,34.90
3,3,1,3,3,3,3,N,3,12.10,7.25,0.5,0.00,0.0,1.0,20.85
4,4,2,4,4,4,4,N,4,11.40,1.00,0.5,3.28,0.0,1.0,19.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,3066761,2,3066761,3066761,3066761,3066761,None,3066761,15.80,0.00,0.5,3.96,0.0,1.0,23.76
3066762,3066762,2,3066762,3066762,3066762,3066762,None,3066762,22.43,0.00,0.5,2.64,0.0,1.0,29.07
3066763,3066763,2,3066763,3066763,3066763,3066763,None,3066763,17.61,0.00,0.5,5.32,0.0,1.0,26.93
3066764,3066764,2,3066764,3066764,3066764,3066764,None,3066764,18.15,0.00,0.5,4.43,0.0,1.0,26.58


In [ ]:
import requests
!pip install boto3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.4 MB/s eta 0:00:00


In [ ]:
import boto3

# Create an S3 client
s3_client = boto3.client('s3')

# Specify the bucket and file key
bucket_name = 'nyc-taxidata-bucket'
file_key = 'yellow_tripdata_2023-01.parquet'

# Retrieve the object from S3
response = s3_client.get_object(Bucket=bucket_name, Key=file_key)

# Access the contents of the object
content = response['Body'].read()

# Process the content as needed
print(content)


NoCredentialsError: ignored